In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

key_training_data = []

class Labeling():        
        
    def make_training_data():
        
        
        keys = ['C','C#','Db','D','D#','Eb','E','F','F#','Gb','G','G#','Ab','A','A#','Bb','B']
        keyIndex = [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]
        keys_counters= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        path = "C:/outs/spects4096"
        #path = "C:/outs/square4096"
        os.chdir(path)
        spectrograms = os.listdir()
        key_count = [0] * 24
        annotations = open("C:/outs/shiftedAnnotations4096.txt", "r")
        #annotations = open("C:/outs/square4096.txt", "r")
        while True: 
            line = annotations.readline()
            if not line:
                break;
            
            
            file, key, chord = line.split(' ', 2)
            key = key.strip()
            chord = chord.strip()
            file = file + ".LOFI.png"
            spect_path = os.path.join(path, file)
            spect_path = cv2.imread(spect_path)
            #spect_path = cv2.resize(spect_path, (224, 224))  
            #spect_path = cv2.imread(spect_path, cv2.IMREAD_GRAYSCALE)
            
            
            if spect_path is None :
                continue;
                
            for j,i in enumerate(keys):
                
                temp = keyIndex[j]
                if keys_counters[temp] > 200 : 
                    continue;
                
                if key == i:
                    key_training_data.append([np.array(spect_path)/255, torch.LongTensor([keyIndex[j]])])
                    keys_counters[temp] += 1
           
            #print(file)
            #for j,i in enumerate(chords):
                #if (chordsCounters[0] > 6500) & (chord == 'minor'):
                    #continue;
                    
                #if (chordsCounters[1] > 2500) :
                    #continue;
                    
                #if chord == i:
                   #chord_training_data.append([np.array(spect_path)/255, torch.LongTensor([chordsIndex[j]])])
                   # chordsCounters[j] += 1

        
        #np.random.shuffle(key_training_data)
        np.random.shuffle(key_training_data)
        print("saving data.....ornah\n")
        #np.save("chord_training_data.npy", chord_training_data)
            
        print(keys_counters)
        
if __name__ == '__main__':
    Labeling.make_training_data()
    
   
    

saving data.....ornah

[201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201]


In [5]:
import torchvision.models as models
import torchvision
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



pretrained = torchvision.models.vgg16(pretrained=True)

ct = 0 
for child in pretrained.children():
    ct += 1 
    if ct < 30:
        for param in child.parameters():
            param.requires_grad = False 

class MyAlexNet(nn.Module):
    def __init__(self, my_pretrained_model):
        super(MyAlexNet, self).__init__()
        self.pretrained = my_pretrained_model
        self.my_new_layers = nn.Sequential(nn.Linear(1000, 100),
                                           nn.ReLU(),
                                           nn.Linear(100, 12))
    
    def forward(self, x):
        x = self.pretrained(x)
        x = self.my_new_layers(x)
        return x

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    my_extended_model = MyAlexNet(my_pretrained_model=pretrained).to(device)

    
my_extended_model 

MyAlexNet(
  (pretrained): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
     

In [6]:
#SETTING OPTIMIZER
import torch.optim as optim


optimizer = torch.optim.SGD(my_extended_model.parameters(), lr=0.001, momentum = 0.9, weight_decay = 0.0001) 
loss_function = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    loss_function = loss_function.to(device)

print("optimizer komple ")

optimizer komple 


In [ ]:
#TRAINING KEY NEURAL NETWORK

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

val_size = int(len(key_training_data)*0.1)

print_flag = True
correct, total = 0, 0
batchSize = 16
training_data = key_training_data[:len(key_training_data)-val_size]
test_data = key_training_data[-val_size:]

test_dataloader = DataLoader(test_data, batch_size = batchSize, shuffle=True, drop_last=True)
train_dataloader = DataLoader(training_data, batch_size=batchSize , shuffle=True, pin_memory=True, drop_last=True)

print("**data loader ftw**")

my_extended_model.train()
num_epochs = 134

#test_counter = 0
for epoch in range(0, num_epochs):
    print("---- Starting new epoch ----")
    #running_loss = 0.0
    #loss_values = [] 
    
    for i, data in enumerate(train_dataloader, 0):
        
        
        #test_counter += 1
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = my_extended_model(inputs)
        
        
        targets = targets.view(len(data[1]))
        
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
    
    if (((epoch+1) % 5) == 0):
        
        print(f"Epoch: {epoch}. Loss: {loss} ")
        print_flag = False
        correct, total = 0, 0
        #k_Net.eval()
        my_extended_model.eval()
        
        print("\n")
        print("``Starting mini Testing``")
        with torch.no_grad():
            #running_loss = 0.0
            #loss_values = [] 

            # Iterate over the test data and generate predictions
            for i, test_data in enumerate(test_dataloader, 0):
            # Get inputs
                test_inputs = test_data[0].view(-1,3,105,600)
                test_targets = test_data[1]
                test_inputs = test_inputs.float()

                test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
            # Generate outputs
                test_outputs = my_extended_model(test_inputs)

            # Set total and correct

                _, predicted = torch.max(test_outputs.data, 1)
                total += test_targets.size(0)
                #running_loss =+ loss.item() * batchSize
                #correct += (predicted == targets).sum().item()
                cntr = 0

                for i in predicted:

                    if i == test_targets[cntr]:
                        correct += 1
                    cntr += 1

                #loss_values.append(running_loss / len(test_data))

        print("Accuracy: ", 100*(correct/total))
        print("\n")
        my_extended_model.train()

        

        #running_loss =+ loss.item() * batchSize
        #loss_values.append(running_loss / len(training_data))
    if print_flag == True:
        print(f"Epoch: {epoch}. Loss: {loss} \n")
    else:
        print_flag = True

**data loader ftw**
---- Starting new epoch ----
Epoch: 0. Loss: 2.4797537326812744 

---- Starting new epoch ----
Epoch: 1. Loss: 2.446108341217041 

---- Starting new epoch ----
Epoch: 2. Loss: 2.4826271533966064 

---- Starting new epoch ----
Epoch: 3. Loss: 2.472019910812378 

---- Starting new epoch ----
Epoch: 4. Loss: 2.4806265830993652 


``Starting mini Testing``
Accuracy:  5.833333333333333


---- Starting new epoch ----
Epoch: 5. Loss: 2.499516248703003 

---- Starting new epoch ----
Epoch: 6. Loss: 2.468791961669922 

---- Starting new epoch ----
Epoch: 7. Loss: 2.4821276664733887 

---- Starting new epoch ----
Epoch: 8. Loss: 2.453643321990967 

---- Starting new epoch ----
Epoch: 9. Loss: 2.4466476440429688 


``Starting mini Testing``
Accuracy:  7.916666666666666


---- Starting new epoch ----
Epoch: 10. Loss: 2.4677200317382812 

---- Starting new epoch ----
Epoch: 11. Loss: 2.4778237342834473 

---- Starting new epoch ----
Epoch: 12. Loss: 2.452038288116455 

---- Star

In [ ]:
from torch.utils.data import DataLoader

correct, total = 0, 0
batchSize = 32
test_data = key_training_data[-val_size:]
test_dataloader = DataLoader(test_data, batch_size = batchSize, shuffle=True, drop_last=True)

#k_Net.eval()
my_extended_model.eval()

print("``````Starting Testing``````")
with torch.no_grad():
    #running_loss = 0.0
    #loss_values = [] 

    # Iterate over the test data and generate predictions
    for i, data in enumerate(test_dataloader, 0):
    # Get inputs
        inputs = data[0].view(-1,3,105,600)
        targets = data[1]
        inputs = inputs.float()

        inputs, targets = inputs.to(device), targets.to(device)
    # Generate outputs
        outputs = my_extended_model(inputs)

    # Set total and correct

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        #running_loss =+ loss.item() * batchSize
        #correct += (predicted == targets).sum().item()
        cntr = 0
        
        for i in predicted:

            if i == targets[cntr]:
                correct += 1
            cntr += 1

        #loss_values.append(running_loss / len(test_data))
    
print("Accuracy: ", 100*(correct/total))

In [ ]:
print(correct)
print(total)
print(len(key_training_data))
print(len(training_data))